In [ ]:
from NewLibraryENG import *

In [ ]:
import glob
list_dir=glob.glob("/Users/Gaia_1/Desktop/h5files/*.h5")

In [ ]:
fs = 10000 #Sampling Frequency
all_data=[]
all_ref=[]
for file in tqdm(list_dir):
    string=file
    data=h5py.File(string,'r')
    data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
    info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
    info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
    labels = info_table['Label']
    readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)
    ref=readings[b'Ref']
    len_data=int(len(readings)/3)
    readings=readings.drop([b'Ref'],axis=1)
    readings=readings.iloc[0:len_data, 17:23]
    ref=ref[0:len_data]
    all_ref.append(ref)
    print(readings.shape)
    all_data.append(readings)

In [ ]:
lowcut = 300
highcut = 3000
fs=10000
order=8
b,a=butter_bandpass(lowcut,highcut,fs,order=order)
filtered_all_data=[]
for file in tqdm(range(len(all_data))):
    prova=all_data[file]
    ref=all_ref[file]
    prova_rows = range(prova.shape[0])
    filt_prova = pd.DataFrame(data = 0, columns=prova.columns, index=prova_rows, dtype = "float32")
    filt_ref=filtfilt(b,a,ref)
    for x in tqdm(range(prova.shape[1])):
        filt_prova.values[:,x] = scipy.signal.filtfilt(b, a, prova.values[:,x])
    for electrode in tqdm(prova.columns):
        filt_prova[electrode] = filt_prova[electrode] - filt_ref
    filtered_all_data.append(filt_prova)
    filt_prova.shape

# Detection

In [ ]:
pos_ind=[]
neg_ind=[]
for file in tqdm(range(len(filtered_all_data))):
    pos_ind_file=[]
    neg_ind_file=[]
    prova=filtered_all_data[file]
    for electrode in tqdm(prova.columns):
        channel=prova[electrode]
        pos, neg=find_all_spikes(channel)
        pos_ind_file.append(pos)
        neg_ind_file.append(neg)
    pos_ind.append(pos_ind_file)
    neg_ind.append(neg_ind_file)

# Cutouts

In [ ]:
pos_cut=[]
neg_cut=[]
n_pos=[]
n_neg=[]
for file in tqdm(range(len(filtered_all_data))):
    pos_cut_file=[]
    neg_cut_file=[]
    n_pos_file=[]
    n_neg_file=[]
    prova=filtered_all_data[file]
    pos_ind_file=pos_ind[file]
    neg_ind_file=neg_ind[file]
    for i,electrode in enumerate(tqdm(prova.columns)):
        pos=pos_ind_file[i]
        neg=neg_ind_file[i]
        channel=prova[electrode]
        pos_cut1,n_pos1, neg_cut1,n_neg1 = cut(pos,neg,channel)
        pos_cut_file.append(pos_cut1)
        neg_cut_file.append(neg_cut1)
        n_pos_file.append(n_pos1)
        n_neg_file.append(n_neg1)
    pos_cut.append(pos_cut_file)
    neg_cut.append(neg_cut_file)
    n_pos.append(n_pos_file)
    n_neg.append(n_neg_file)
#savedp = copy.deepcopy(pos_cut)
#savedn = copy.deepcopy(neg_cut)

In [ ]:
for file in tqdm(range(len(filtered_all_data))):
    pos_cut_file=pos_cut[file]
    neg_cut_file=neg_cut[file]
    for i in (tqdm(range(len(pos_cut_file)))):
        pos_cut_file[i]=mask_cuts(pos_cut_file[i])
        neg_cut_file[i]= mask_cuts(neg_cut_file[i])
    pos_cut[file]=pos_cut_file
    neg_cut[file]=neg_cut_file

# Clustering

In [ ]:
pos_clusters=[]
for file in tqdm(range(len(filtered_all_data))):
    data=filtered_all_data[file]
    pos_cut_file=pos_cut[file]
    n_pos_file=n_pos[file]
    final_data_pos=[]
    for channel in (tqdm(range(len(pos_cut)))):
        channel_clusters=clus(pos_cut_file[channel],'kmeans',n_pos_file[channel],data)
        final_data_pos.append(channel_clusters)
    pos_clusters.append(final_data_pos)

In [ ]:
neg_clusters=[]
for file in tqdm(range(len(filtered_all_data))):
    data=filtered_all_data[file]
    neg_cut_file=neg_cut[file]
    n_neg_file=n_neg[file]
    final_data_neg=[]
    for channel in (tqdm(range(len(neg_cut_file)))):
        channel_clusters=clus(neg_cut_file[channel],'kmeans',n_neg_file[channel],data)
        final_data_neg.append(channel_clusters)
    neg_clusters.append(final_data_neg)

In [ ]:
final_data_pos=pos_clusters[0]
final_data_pos
#final_data_pos[electrode]
electrode=final_data_pos[0]
len(electrode)
electrode
#neuron=electrode[0]
#len(neuron)
#neuron

## capire come salvare i neuroni clusterizzati e usarli per point process

In [ ]:
neg_clusters[0][0]

In [ ]:
neurons = []
for file in pos_clusters:    
    for electrode in file:
        neuron_clus=[]
        neuron_elec=[]
        for cluster in electrode:
            for neuron in cluster:
                neuron_clus.append(neuron)
            neuron_elec.append(neuron_clus)
    neurons.append(neuron_elec)

len(neurons)

In [ ]:
for file in neg_clusters:
    for electrode in file:
        neuron_clus=[]
        neuron_elec=[]
        for cluster in electrode:
            for neuron in cluster:
                neuron_clus.append(neuron)
            neuron_elec.append(neuron_clus)
    neurons.append(neuron_elec)
len(neurons)

In [ ]:
s=0
for i in range(len(neurons)):
    s=s+len(neurons[i])
    #print(len(neurons[i]))
    for j in range(len(neurons[i])):
        #print(len(neurons[i][j]))
        s=s+len(neurons[i][j])
s

# Export (after sorting)

In [ ]:
for file in list_dir:
    name_data=file
    save_data = 'After'+name_data+'.txt'
    np.savetxt(r"C:\After_SS\%save_data.txt" % count,adj_neur[file], delimiter=', ', fmt='%12.8f')

# Point Process

In [ ]:
adj_neur=[]
counter = 0
max_len=0

for neu in neurons:
    print('counter: ',counter,neu.shape[0])
    if neu.shape[0]>max_len:
        max_len=neu.shape[0]
    counter+=1
for neuron in neurons:
    if neuron.shape[0]<=max_len:
        diff = max_len-neuron.shape[0]
        adj_neur.append(np.concatenate((neuron,np.zeros([diff]))))


In [ ]:
len(neurons[0][1])

In [ ]:
adj_neur=[]
counter = 0
max_len=0
for file in neurons:
    for clus in file:
        for neu in clus:
            print('counter: ',counter,neu.shape[0])
            if neu.shape[0]>max_len:
                max_len=neu.shape[0]
        counter+=1
    for clus in file:
        for neu in clus:
            if neuron.shape[0]<=max_len:
                diff = max_len-neuron.shape[0]
                adj_neur.append(np.concatenate((neuron,np.zeros([diff]))))
